# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

#data files
cities_data = "../output_data/cities_data.csv"


# Read the csv file
cities = pd.read_csv(cities_data, index_col=0)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
print(type(cities))


<class 'pandas.core.frame.DataFrame'>


In [3]:
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Staritsa,98,RU,1601608300,73,56.51,34.94,8.63,2.95
1,Bluff,100,NZ,1601608184,59,-46.60,168.33,14.44,4.02
2,Klaksvík,100,FO,1601608192,100,62.23,-6.59,11.00,10.80
3,Rikitea,0,PF,1601608180,79,-23.12,-134.97,22.94,8.01
4,Mataura,100,NZ,1601608195,59,-46.19,168.86,14.44,4.02
...,...,...,...,...,...,...,...,...,...
508,Udachny,75,RU,1601608205,68,66.42,112.40,-4.00,3.00
509,Hay River,75,CA,1601608533,93,60.82,-115.80,7.00,3.10
510,Behbahān,0,IR,1601608534,65,30.60,50.24,25.00,2.10
511,Nadym,100,RU,1601608534,81,65.53,72.52,6.65,3.55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)
# store humidity to be used as weight - to float in df then as singular for weight to make it a float
humid_lat = cities[["Humidity", "Lat"]].astype(float)
humidity = humid_lat["Humidity"].astype(float)

In [6]:
# Create Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Narrow down the DataFrame for my ideal weather conditions. 
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#max temperature between -4 and 12 degrees C, Wind speed >3 m/s and cloudiness <= 30 
holiday_city_df = cities.loc[(cities["Wind Speed"] >= 8) & (cities["Cloudiness"] >= 80) & \
                             (cities["Max Temp"] >= 10) & (cities["Max Temp"] <= 15)].dropna()

print("There are "+str(len(holiday_city_df))+" cities with my weather prefernces for a holiday")


There are 10 cities with my weather prefernces for a holiday


In [8]:
holiday_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,Klaksvík,100,FO,1601608192,100,62.23,-6.59,11.00,10.80
17,East London,100,ZA,1601608189,73,-33.02,27.91,14.90,8.01
66,Port Alfred,100,ZA,1601608203,72,-33.59,26.89,14.39,9.95
139,Port Elizabeth,90,ZA,1601608198,58,-33.92,25.57,15.00,9.80
150,Bredasdorp,83,ZA,1601608233,58,-34.53,20.04,14.00,9.30
159,Saint Paul Harbor,90,US,1601608177,81,57.79,-152.41,11.00,12.30
190,Foz,97,ES,1601608457,1,43.57,-7.25,13.33,8.03
287,Homer,100,US,1601608480,84,59.64,-151.55,10.00,8.05
333,Brae,96,GB,1601608268,82,60.40,-1.35,13.00,11.80
353,Châtellerault,100,FR,1601608495,93,46.80,0.53,13.33,9.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#create hotel_df with holiday's cities and add column for hotel name
hotel_df = holiday_city_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,Klaksvík,100,FO,1601608192,100,62.23,-6.59,11.00,10.80,
17,East London,100,ZA,1601608189,73,-33.02,27.91,14.90,8.01,
66,Port Alfred,100,ZA,1601608203,72,-33.59,26.89,14.39,9.95,
139,Port Elizabeth,90,ZA,1601608198,58,-33.92,25.57,15.00,9.80,
150,Bredasdorp,83,ZA,1601608233,58,-34.53,20.04,14.00,9.30,
159,Saint Paul Harbor,90,US,1601608177,81,57.79,-152.41,11.00,12.30,
190,Foz,97,ES,1601608457,1,43.57,-7.25,13.33,8.03,
287,Homer,100,US,1601608480,84,59.64,-151.55,10.00,8.05,
333,Brae,96,GB,1601608268,82,60.40,-1.35,13.00,11.80,
353,Châtellerault,100,FR,1601608495,93,46.80,0.53,13.33,9.30,


In [10]:
#set up googlemaps api url and paramaters for calling this api
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [11]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # add to url and call API 
    print(f"Searching for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # put response into results
    results = response['results']
    
    # parse the name of the hotel into the hotel df dataframe
    try:
        print(f"Centrally located hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel found... Looking for Hotel in next city.")
        
    print("------------")
    

# Print end of search once searching is completed
print("-------End of Search-------")

Searching for Index 2: Klaksvík.
Centrally located hotel in Klaksvík is Hotel Klaksvík.
------------
Searching for Index 17: East London.
Centrally located hotel in East London is Premier Hotel El Icc.
------------
Searching for Index 66: Port Alfred.
Centrally located hotel in Port Alfred is Royal St Andrews Hotel.
------------
Searching for Index 139: Port Elizabeth.
Centrally located hotel in Port Elizabeth is Lemon Tree Lane Guest House.
------------
Searching for Index 150: Bredasdorp.
Centrally located hotel in Bredasdorp is De Hoop Collection.
------------
Searching for Index 159: Saint Paul Harbor.
Centrally located hotel in Saint Paul Harbor is Best Western Kodiak Inn and Convention Center.
------------
Searching for Index 190: Foz.
Centrally located hotel in Foz is Hotel Villa De Foz.
------------
Searching for Index 287: Homer.
Centrally located hotel in Homer is Seafarer Suites.
------------
Searching for Index 333: Brae.
Centrally located hotel in Brae is Busta House Hotel

In [12]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,Klaksvík,100,FO,1601608192,100,62.23,-6.59,11.00,10.80,Hotel Klaksvík
17,East London,100,ZA,1601608189,73,-33.02,27.91,14.90,8.01,Premier Hotel El Icc
66,Port Alfred,100,ZA,1601608203,72,-33.59,26.89,14.39,9.95,Royal St Andrews Hotel
139,Port Elizabeth,90,ZA,1601608198,58,-33.92,25.57,15.00,9.80,Lemon Tree Lane Guest House
150,Bredasdorp,83,ZA,1601608233,58,-34.53,20.04,14.00,9.30,De Hoop Collection
159,Saint Paul Harbor,90,US,1601608177,81,57.79,-152.41,11.00,12.30,Best Western Kodiak Inn and Convention Center
190,Foz,97,ES,1601608457,1,43.57,-7.25,13.33,8.03,Hotel Villa De Foz
287,Homer,100,US,1601608480,84,59.64,-151.55,10.00,8.05,Seafarer Suites
333,Brae,96,GB,1601608268,82,60.40,-1.35,13.00,11.80,Busta House Hotel
353,Châtellerault,100,FR,1601608495,93,46.80,0.53,13.33,9.30,La Gourmandine


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))